In [1]:
!pip install transformers evaluate datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.4/492.4 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.4 MB/s eta 0:00:00


In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.8 MB/s eta 0:00:00


In [3]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer # baseline code용 tfidef vectorirzer(또는 tokenizer)
from sklearn import preprocessing
from sklearn.metrics import f1_score
from IPython.display import display, HTML

from transformers import AutoModel, AutoTokenizer,AutoModelForSequenceClassification, TrainingArguments, Trainer # 사용하고자 하는 모델, 토크나이저 적용시 필요
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset, ClassLabel

from tqdm.auto import tqdm # process bar 표시용

import warnings
warnings.filterwarnings(action='ignore')

import urllib.request

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device  # pt 코드에서 필요함

device(type='cuda')

In [6]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")
train_data = pd.read_table('ratings_train.txt')
val_data = pd.read_table('ratings_test.txt')

In [7]:
pretrained_model = 'klue/roberta-small'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

In [8]:
test_model = AutoModel.from_pretrained(pretrained_model)
test_model

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-5): 6 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout)

In [13]:
sentences = ["무더운 여름에 건강관리 잘하세요!"]
features = tokenizer(
    sentences,
    max_length=20,
    padding="max_length",
    truncation=True,
)
features

{'input_ids': [[0, 21972, 4565, 2170, 3996, 7604, 1521, 2205, 5971, 5, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}

In [14]:
train_data = train_data.dropna()
val_data = val_data.dropna()

In [16]:
# 딕셔너리 구조를 사용하여 학습용과 검증용 데이터를 쉽게 분류하여 활용
from datasets import Dataset, DatasetDict

tds = Dataset.from_pandas(train_data) #  datasets.Dataset 객체로 변환하여 데이터를 로드
vds = Dataset.from_pandas(val_data)

ds = DatasetDict() # datasets.Dataset 객체를 관리하는 딕셔너리 구조를 제공

ds['train'] = tds
ds['validation'] = vds

In [17]:
# 데이터 tokenize
def preprocess_function(examples): # 데이터셋을 토큰화하는 함수
    return tokenizer(examples["document"], truncation=True,  return_token_type_ids=False,)

# ds라는 데이터셋을 preprocess_function 함수를 사용하여 토큰화한 결과를 저장
tokenized_datasets = ds.map(preprocess_function, batched=True)



Map:   0%|          | 0/149995 [00:00<?, ? examples/s]

Map:   0%|          | 0/49997 [00:00<?, ? examples/s]

In [18]:
# loss type define
import evaluate
accuracy = evaluate.load("accuracy")

In [20]:
# 모델의 평가 결과인 eval_pred를 인자로 받아서 정확도를 계산하여 반환
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1) # 각 샘플의 두번째 차원을 따라 최대값을 갖는 인덱스를 반환
    return accuracy.compute(predictions=predictions, references=labels)

id2label = {0: "부정", 1: "긍정" }
label2id = {"부정":0, "긍정":1}

In [22]:
# hugging face의 transformers 라이브러리를 사용하여 사전학습된 모델을 불러오고 sequence classification 작업에 활용할 수 있는 모델 객체를 생성
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model, num_labels=2, id2label=id2label, label2id=label2id)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### Training Arguments 설정
- output_dir: 학습된 모델과 기타 출력 파일들을 저장할 경로를 지정합니다. 여기서는 "my_model" 디렉토리에 저장합니다.

- learning_rate: 학습 속도를 결정하는 학습률입니다. 2e-5는 0.00002를 의미하며, 학습 과정에서 가중치 업데이트에 사용됩니다.

- per_device_train_batch_size: 모델 학습 시 한 번에 처리할 훈련 데이터의 배치 크기입니다. 16개의 토큰으로 이루어진 배치를 사용합니다.

- per_device_eval_batch_size: 모델 평가 시 한 번에 처리할 검증 데이터의 배치 크기입니다. 훈련과 마찬가지로 16개의 토큰으로 이루어진 배치를 사용합니다.

- num_train_epochs: 전체 훈련 데이터셋을 몇 번 반복해서 학습할지를 결정하는 에포크(epoch) 수입니다. 여기서는 1 에포크를 수행합니다.

- weight_decay: L2 정규화를 위한 가중치 감소 비율을 설정합니다. 0.01로 설정되어 있으므로, 가중치를 1% 만큼 감소시킵니다.

- evaluation_strategy: 평가 전략을 결정합니다. "epoch"로 설정되어 있으므로, 각 에포크의 끝에서 평가가 수행됩니다.

- save_strategy: 모델 저장 전략을 결정합니다. "epoch"로 설정되어 있으므로, 각 에포크의 끝에서 학습된 모델을 저장합니다.

- load_best_model_at_end: 학습이 완료된 후 가장 성능이 좋은 모델을 로드할지를 결정합니다. True로 설정되어 있으므로, 학습이 완료된 후 최상의 모델을 불러옵니다.

#### Trainer 설정
- model: 학습시킬 모델을 지정합니다. model은 이미 초기화된 모델 객체입니다.

- args: 앞에서 설정한 TrainingArguments 객체를 사용하여 학습 설정을 지정합니다.

- train_dataset: 훈련에 사용할 데이터셋을 지정합니다. 여기서는 "tokenized_datasets"라는 사전에 저장된 토큰화된 훈련 데이터를 사용합니다.

- eval_dataset: 검증에 사용할 데이터셋을 지정합니다. 마찬가지로 "tokenized_datasets" 사전에 저장된 토큰화된 검증 데이터를 사용합니다.

- tokenizer: 토크나이저 객체를 지정합니다. 이 토크나이저는 모델의 입력 데이터를 토큰화하는 데 사용됩니다.

- compute_metrics: 평가 시 사용할 메트릭을 정의하는 함수입니다. 평가 결과를 평가하기 위해 사용자 정의 메트릭을 정의할 수 있습니다.

In [23]:
training_args = TrainingArguments(
    output_dir="my_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.251600,0.257048,0.898274


TrainOutput(global_step=9375, training_loss=0.29368047607421877, metrics={'train_runtime': 1086.6312, 'train_samples_per_second': 138.037, 'train_steps_per_second': 8.628, 'total_flos': 2452356532499904.0, 'train_loss': 0.29368047607421877, 'epoch': 1.0})

In [24]:
trainer.evaluate()

{'eval_loss': 0.2570481300354004,
 'eval_accuracy': 0.8982738964337861,
 'eval_runtime': 90.0247,
 'eval_samples_per_second': 555.37,
 'eval_steps_per_second': 34.713,
 'epoch': 1.0}

In [32]:
text ='영화시간 내내 긴장을 늦출수가 없었어요!!!'
inputs =  tokenizer(text, truncation=True,  return_token_type_ids=False, return_tensors="pt").to(device)
model.eval()
outputs = model(**inputs)
predictions = outputs.logits.argmax(-1)
id2label[predictions.cpu().numpy()[0]]

'긍정'

In [33]:
outputs.logits

tensor([[-2.2553,  2.1561]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [34]:
# Save the model manually
# config.json 및 pytorch_model.bin 파일이 "my_model" 디렉토리에 생성
model.save_pretrained("my_model")

In [ ]:
import os
os.listdir("my_model")

['pytorch_model.bin', 'runs', 'checkpoint-9375', 'config.json']

In [35]:
!ls

drive  my_model  ratings_test.txt  ratings_train.txt  sample_data


In [36]:
!mkdir ./drive/MyDrive/hmkd1/강의/nlp/best_model

In [37]:
!cp -r ./my_model ./drive/MyDrive/hmkd1/강의/nlp/best_model